In [1]:
!pip install groq openai

In [2]:
import json
import re
from typing import List, Dict, Any, Optional
from groq import Groq
from google.colab import userdata

api_key = userdata.get('GROQ_API_KEY')
client = Groq(api_key=api_key)

class ConversationManager:
    def __init__(self, max_turns: int = None, max_length: int = None, summarize_every: int = None):
        self.history = []
        self.max_turns = max_turns
        self.max_length = max_length
        self.summarize_every = summarize_every
        self.turn_count = 0

    def add_message(self, role: str, content: str):
        self.history.append({"role": role, "content": content})
        self.turn_count += 1

        if self.summarize_every and self.turn_count % self.summarize_every == 0:
            self.summarize_conversation()

        self._apply_truncation()

    def _apply_truncation(self):
        if self.max_turns and len(self.history) > self.max_turns:
            self.history = self.history[-self.max_turns:]

        if self.max_length:
            current_length = sum(len(msg["content"]) for msg in self.history)
            while current_length > self.max_length and len(self.history) > 1:
                removed = self.history.pop(0)
                current_length -= len(removed["content"])

    def summarize_conversation(self):
        if len(self.history) < 2:
            return

        summary_prompt = [
            {"role": "system", "content": "You are a helpful assistant that summarizes conversations. Create a concise summary of the key points discussed so far."},
            {"role": "user", "content": f"Please summarize this conversation:\n\n{self._format_history()}"}
        ]

        try:
            response = client.chat.completions.create(
                model="llama-3.1-8b-instant",
                messages=summary_prompt,
                temperature=0.3,
                max_tokens=500
            )

            summary = response.choices[0].message.content

            self.history = [
                {"role": "system", "content": f"Previous conversation summary: {summary}"}
            ]

        except Exception as e:
            print(f"Error during summarization: {e}")

    def _format_history(self) -> str:
        return "\n".join([f"{msg['role']}: {msg['content']}" for msg in self.history])

    def get_history(self) -> List[Dict[str, str]]:
        return self.history.copy()

    def clear_history(self):
        self.history = []
        self.turn_count = 0

    def display_history(self, title: str = "Conversation History"):
        print("=" * 60)
        print(title)
        print("=" * 60)
        for i, msg in enumerate(self.history, 1):
            print(f"{i}. {msg['role']}: {msg['content']}")
        print()

print("=" * 60)
print("TASK 1: CONVERSATION HISTORY MANAGEMENT")
print("=" * 60)

conversation_samples = [
    ("user", "Hi, my name is John Doe. I'm interested in learning more about your services."),
    ("assistant", "Hello John! I'd be happy to tell you about our services. What specifically are you looking for?"),
    ("user", "I need help with web development for my small business. I'm based in New York."),
    ("assistant", "We offer comprehensive web development services. Could you tell me more about your business?"),
    ("user", "I run a boutique coffee shop and need an online ordering system. My email is john.doe@email.com."),
    ("assistant", "An e-commerce solution for a coffee shop sounds great! What's your timeline for this project?"),
    ("user", "I'd like to launch in about 3 months. By the way, I'm 35 years old."),
    ("assistant", "That's a reasonable timeline. We can definitely work with that."),
    ("user", "My phone number is 555-123-4567. When can we schedule a consultation?"),
    ("assistant", "I can have our project manager reach out to you tomorrow. Does that work?")
]

test_cases = [
    {"max_turns": 4, "max_length": None, "summarize_every": None},
    {"max_turns": None, "max_length": 300, "summarize_every": None},
    {"max_turns": 6, "max_length": None, "summarize_every": 3}
]

for i, settings in enumerate(test_cases, 1):
    print(f"\nTest Case {i}: Settings: {settings}")
    print("-" * 40)

    manager = ConversationManager(
        max_turns=settings["max_turns"],
        max_length=settings["max_length"],
        summarize_every=settings["summarize_every"]
    )

    for role, content in conversation_samples:
        manager.add_message(role, content)
        print(f"Added: {role}: {content[:50]}{'...' if len(content) > 50 else ''}")
        manager.display_history(f"History for Test Case {i}")
        if len(manager.history) <= 3:
            print(f"History length: {len(manager.history)} messages")

    manager.display_history(f"Final History for Test Case {i}")

TASK 1: CONVERSATION HISTORY MANAGEMENT

Test Case 1: Settings: {'max_turns': 4, 'max_length': None, 'summarize_every': None}
----------------------------------------
Added: user: Hi, my name is John Doe. I'm interested in learnin...
History for Test Case 1
1. user: Hi, my name is John Doe. I'm interested in learning more about your services.

History length: 1 messages
Added: assistant: Hello John! I'd be happy to tell you about our ser...
History for Test Case 1
1. user: Hi, my name is John Doe. I'm interested in learning more about your services.
2. assistant: Hello John! I'd be happy to tell you about our services. What specifically are you looking for?

History length: 2 messages
Added: user: I need help with web development for my small busi...
History for Test Case 1
1. user: Hi, my name is John Doe. I'm interested in learning more about your services.
2. assistant: Hello John! I'd be happy to tell you about our services. What specifically are you looking for?
3. user: I need he

In [3]:
INFO_EXTRACTION_SCHEMA = {
    "name": "extract_user_info",
    "description": "Extract user information from the conversation",
    "parameters": {
        "type": "object",
        "properties": {
            "name": {
                "type": "string",
                "description": "The user's full name"
            },
            "email": {
                "type": "string",
                "description": "The user's email address"
            },
            "phone": {
                "type": "string",
                "description": "The user's phone number"
            },
            "location": {
                "type": "string",
                "description": "The user's location/city"
            },
            "age": {
                "type": "integer",
                "description": "The user's age"
            }
        },
        "required": ["name", "email", "phone", "location", "age"]
    }
}

class InformationExtractor:
    def __init__(self, schema: Dict[str, Any] = INFO_EXTRACTION_SCHEMA):
        self.schema = schema

    def extract_information(self, conversation: List[Dict[str, str]]) -> Dict[str, Any]:
        try:
            messages = conversation.copy()
            messages.append({
                "role": "user",
                "content": "Please extract the user information from this conversation."
            })

            response = client.chat.completions.create(
                model="llama-3.1-8b-instant",
                messages=messages,
                functions=[self.schema],
                function_call={"name": "extract_user_info"},
                temperature=0.1
            )

            if response.choices[0].message.function_call:
                function_call = response.choices[0].message.function_call
                if function_call.name == "extract_user_info":
                    return json.loads(function_call.arguments)

            return {}

        except Exception as e:
            print(f"Error during information extraction: {e}")
            return {}

    def validate_extracted_info(self, info: Dict[str, Any]) -> bool:
        required_fields = self.schema["parameters"]["properties"].keys()
        for field in required_fields:
            if field not in info:
                return False

            if field == "email" and info[field]:
                email_regex = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
                if not re.match(email_regex, info[field]):
                    return False

            if field == "phone" and info[field]:
                digits = re.sub(r'\D', '', info[field])
                if len(digits) < 10:
                    return False

            if field == "age" and info[field]:
                try:
                    age = int(info[field])
                    if age < 0 or age > 120:
                        return False
                except ValueError:
                    return False

        return True

    def display_extracted_info(self, info: Dict[str, Any], title: str = "Extracted Information"):
        print("=" * 60)
        print(title)
        print("=" * 60)
        for key, value in info.items():
            print(f"{key.capitalize()}: {value}")
        print(f"Validation: {'PASS' if self.validate_extracted_info(info) else 'FAIL'}")
        print()

print("\n" + "=" * 60)
print("TASK 2: INFORMATION EXTRACTION")
print("=" * 60)

sample_chats = [
    [
        {"role": "user", "content": "Hi, I'm Jane Smith. I'm 28 years old and I live in San Francisco."},
        {"role": "assistant", "content": "Nice to meet you, Jane! How can I help you today?"},
        {"role": "user", "content": "I'd like to sign up for your newsletter. My email is jane.smith@example.com and my phone is 415-555-1234."}
    ],
    [
        {"role": "user", "content": "My name is Robert Johnson, I'm from Chicago, and I'm 42."},
        {"role": "assistant", "content": "Hello Robert! What brings you to our service today?"},
        {"role": "user", "content": "I need support with my account. You can reach me at robj@company.com or 312-555-9876."}
    ],
    [
        {"role": "user", "content": "I'm Emily Davis, 31 years old, living in Boston."},
        {"role": "assistant", "content": "Welcome Emily! How can I assist you?"},
        {"role": "user", "content": "I want to update my contact information. My new email is emily.d@email.org and my phone is 617-555-2468."}
    ]
]

extractor = InformationExtractor()

for i, chat in enumerate(sample_chats, 1):
    print(f"\nSample Chat {i}:")
    print("-" * 30)

    for msg in chat:
        print(f"{msg['role']}: {msg['content']}")

    extracted_info = extractor.extract_information(chat)
    extractor.display_extracted_info(extracted_info, f"Extracted Information from Chat {i}")

print("Demonstration complete!")


TASK 2: INFORMATION EXTRACTION

Sample Chat 1:
------------------------------
user: Hi, I'm Jane Smith. I'm 28 years old and I live in San Francisco.
assistant: Nice to meet you, Jane! How can I help you today?
user: I'd like to sign up for your newsletter. My email is jane.smith@example.com and my phone is 415-555-1234.
Extracted Information from Chat 1
Age: 28
Email: jane.smith@example.com
Location: San Francisco
Name: Jane Smith
Phone: 415-555-1234
Validation: PASS


Sample Chat 2:
------------------------------
user: My name is Robert Johnson, I'm from Chicago, and I'm 42.
assistant: Hello Robert! What brings you to our service today?
user: I need support with my account. You can reach me at robj@company.com or 312-555-9876.
Extracted Information from Chat 2
Age: 42
Email: robj@company.com
Location: Chicago
Name: Robert Johnson
Phone: 312-555-9876
Validation: PASS


Sample Chat 3:
------------------------------
user: I'm Emily Davis, 31 years old, living in Boston.
assistant: Welc